In [ ]:
# Do all the initialization of objects that don't change

# Define the x-axis for the plot
$xaxis = [XPlot.Plotly.Graph+Xaxis]::new()
$xaxis.nticks = 23
# https://github.com/d3/d3-time-format/blob/master/README.md
$xaxis.tickformat = '%m-%d'

# Define the y-axis for the plot
$yaxis = [XPlot.Plotly.Graph+Yaxis]::new()

# Define the marker for a point
$marker = [XPlot.Plotly.Graph+Marker]::new()
$marker.color = "rgb(255, 0, 0)"
$marker.Size =15

# Define the marker for a point
$recoveredMarker = [XPlot.Plotly.Graph+Marker]::new()
$recoveredMarker.color = "rgb(0, 255, 0)"
$recoveredMarker.Size =10

# Define the marker for a point
$mortalityMarker = [XPlot.Plotly.Graph+Marker]::new()
$mortalityMarker.color = "rgb(0, 0, 0)"
$mortalityMarker.Size =10

# Define the marker for a point
$activeMarker = [XPlot.Plotly.Graph+Marker]::new()
$activeMarker.color = "rgb(255, 165, 0)"
$activeMarker.Size =5

In [ ]:
# Consider switching to https://ourworldindata.org/coronavirus-source-data due to licensing

$confirmedString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv).Content
$recoveredString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv).Content
$mortalityString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv).Content

$usConfirmedAll = $confirmedString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US' }
$usRecoveredAll  = $recoveredString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US' }
$usMortalityAll  = $mortalityString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US' }

$usAllData = [pscustomobject] @{
    Confirmed = $usConfirmedAll
    Recovered = $usRecoveredAll
    Mortality = $usMortalityAll
}

Write-Verbose "Done" -Verbose

In [ ]:
# Filter to the United States

#Filter out state summaries,
$waCityFilterScriptBlock = { ($_.'Province/State' -match '.*, WA' )}
$waStateFilterScriptBlock = { ($_.'Province/State' -match 'Washington$' )}

Function Get-Summary {
    param(
        [scriptBlock] $CityFilterScriptBlock = { $_.'Country/Region' -eq 'US' -and ($_.'Province/State' -match '.*, \w{2}' -or $_.'Province/State' -match '.*Princess' )},
        [ScriptBlock] $StateFilterScriptBlock = { $_.'Country/Region' -eq 'US' -and ($_.'Province/State' -notmatch '.*, \w{2}' -or $_.'Province/State' -match '.*Princess' )},
        [Object] $UsAllData
    )

    $usConfirmedByCity = $usConfirmedAll | Where-Object $cityFilterScriptBlock
    $usRecoveredByCity  = $usRecoveredAll | Where-Object $cityFilterScriptBlock
    $usMortalityByCity  = $usMortality | Where-Object $cityFilterScriptBlock
    $usConfirmedByState = $usConfirmedAll | Where-Object $stateFilterScriptBlock
    $usRecoveredByState  = $usRecoveredAll | Where-Object $stateFilterScriptBlock
    $usMortalityByState  = $usMortality | Where-Object $stateFilterScriptBlock
    
    Write-Verbose "$($usConfirmedByCity.Count) confirmed cases Cities" -verbose
    Write-Verbose "$($usRecoveredByCity.Count) recovery Cities" -verbose
    Write-Verbose "$($usMortalityByCity.Count) mortality Cities" -verbose
    Write-Verbose "$($usConfirmedByState.Count) confirmed cases States/Ships" -verbose
    Write-Verbose "$($usRecoveredByState.Count) recovery States/Ships" -verbose
    Write-Verbose "$($usMortalityByState.Count) mortality States/Ships" -verbose

    # Get unique dates on or after 2/15
    $dates = $usConfirmedByCity[0] | Get-member -MemberType NoteProperty  | Where-Object {$_.Name -match '\d+/\d+/\d+' -and ([datetime] $_.Name) -ge ([datetime] '2/15/2020')} | Select-Object -ExpandProperty Name | ForEach-Object {
            $date = [datetime] $_
            [pscustomobject]@{
                Name = $_
                Date = $date
            }
        } | Sort-Object -Property Date
    #$dates[-1]

    $sum = ($usConfirmedByState | Where-Object {$_.'Province/State' -eq 'Washington'})."$($dates[-1].Name)" | measure-object -sum | select-object -Property Count, Sum
    Write-Verbose "$($dates[-1].Name) washington state confirmed ($($sum.count)) record: $($sum.Sum)  " -verbose
    Write-Verbose "of $($dates.count) days" -verbose
    
    # Summarize the data for each date
    $usSummary = @()
    $dateBoundary = [datetime]'3/9/2020'
    foreach($dateObject in $dates)
    {
        $date = $dateObject.Date
        $dateString = $dateObject.Name

        <#
        <= 3/9 cities are more accurate
        > 3/9 states summarise are more accurate.
        #>
        if($date -le $dateBoundary) {
            $confirmedSum = ($usConfirmedByCity | Measure-Object -Sum -Property $dateString).Sum
            $recoveredSum = ($usRecoveredByCity | Measure-Object -Sum -Property $dateString).Sum
            $mortalitySum = ($usMortalityByCity | Measure-Object -Sum -Property $dateString).Sum
        }
        else
        {
            $confirmedSum = ($usConfirmedByState | Measure-Object -Sum -Property $dateString).Sum
            $recoveredSum = ($usRecoveredByState | Measure-Object -Sum -Property $dateString).Sum
            $mortalitySum = ($usMortalityByState | Measure-Object -Sum -Property $dateString).Sum    
        }

        $usSummary += [pscustomobject] @{
            date = [datetime] $date
            confirmed = $confirmedSum
            recovered = $recoveredSum
            deaths = $mortalitySum
            activeCases = $confirmedSum - ($mortalitySum + $recoveredSum)
        }
    }
    $usSummary = $usSummary | Sort-Object -Property Date
    Write-Verbose "$($usSummary.Count) days" -verbose
    return $usSummary
}

$usSummary = Get-Summary -UsAllData $usAllData

$waSummary = Get-Summary -UsAllData $usAllData -CityFilterScriptBlock $waCityFilterScriptBlock -StateFilterScriptBlock $waStateFilterScriptBlock


In [ ]:
#Format the traces
function Out-SummaryChart 
{
    param(
        [object] $areaSummary,
        [string] $area
    )
    $usConfirmedTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usConfirmedTrace.x  = $areaSummary.Date
    $usConfirmedTrace.y  = $areaSummary.Confirmed
    $usConfirmedTrace.text = 'Confirmed'
    $usConfirmedTrace.marker  = $marker
    $usConfirmedTrace.name = "$area Confirmed"

    $usRecoveredTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usRecoveredTrace.x  = $areaSummary.Date
    $usRecoveredTrace.y  = $areaSummary.Recovered
    $usRecoveredTrace.text = 'Recovered'
    $usRecoveredTrace.marker  = $recoveredMarker
    $usRecoveredTrace.name = "$area Recovered" 

    $usMortalityTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usMortalityTrace.x  = $areaSummary.Date
    $usMortalityTrace.y  = $areaSummary.Deaths
    $usMortalityTrace.text = 'Deaths'
    $usMortalityTrace.marker  = $mortalityMarker
    $usMortalityTrace.name = "$area Deaths"

    $usActiveTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usActiveTrace.x  = $areaSummary.Date
    $usActiveTrace.y  = $areaSummary.ActiveCases
    $usActiveTrace.marker  = $activeMarker
    $usActiveTrace.name = "$area Active Cases"

    $max = @(($areaSummary.ActiveCases | measure-object -max).Maximum, ($areaSummary.Confirmed | measure-object -max).Maximum)
    $yaxis.dtick = [int] (($max | measure-object -max).Maximum / 5)
    
    # Create the chart
    $layout = [XPlot.Plotly.Layout+Layout]::new()
    $layout.width = 1024
    $layout.height = 400

    $layout.xaxis = $xaxis
    $layout.yaxis = $yaxis
    $layout.title = "John Hopkins - $Area - Coronavirus Data"
    $layout.showlegend = $true
    # Render the chart
    New-PlotlyChart -trace $usConfirmedTrace,$usRecoveredTrace,$usMortalityTrace, $usActiveTrace -Layout $layout  | Out-Display
}


In [ ]:
Out-SummaryChart -AreaSummary $usSummary -area 'US'

In [ ]:
Out-SummaryChart -AreaSummary $waSummary -area 'WA'

In [ ]:
<# Used to verify data
 $usConfirmedByCity | ForEach-Object { $parts = $_.'Province/State' -split ', '; $_ | Add-Member -MemberType NoteProperty -Name State -Value $parts[1] -ErrorAction ignore; $_ | Add-Member -MemberType NoteProperty -Name City -Value $parts[0] -ErrorAction ignore; $_ } | 
    Select-Object -Property city, state, '3/11/20'| group-object -Property state | ForEach-Object { $_.Group 
        | measure-Object -Property '3/11/20' -Sum
            | Add-Member -memberType NoteProperty -Name name -value $_.Name -PassThru } | select-Object -Property Name, Count, Sum
#>

In [ ]:
# Used to verify data
# $usConfirmedByState | Select-Object -Property 'Province/State', '3/11/20' -Unique | Sort-Object -Property 'Province/State'